In [1]:
from openai import OpenAI
import pathlib
import textwrap
import os
import time
from pylatexenc.latexencode import unicode_to_latex
from tqdm.notebook import tqdm
from IPython.display import display
from IPython.display import Markdown
import json

In [4]:
questions = []
with open("./batch_responses/tf_197_content_id.jsonl", "r") as file:
    for line in file:
        json_object = json.loads(line)
        custom_id = json_object['custom_id'].split('|')
        q = json_object['response']['body']['choices'][0]["message"]["content"]
        q = q.replace("`", "").replace("json", "").replace("\n", "").replace("\\", "\\\\")
        q = q[q.find("{"):]
        q = q.replace(",}", "}")
        try:
            json_question = json.loads(q)
        except:
            print(q)
            break
        json_question['chapter'] = custom_id[0]
        json_question['section'] = custom_id[1]
        json_question['number'] = custom_id[2]
        questions.append(json_question)


In [5]:
questions

[{'question': 'True or False: If $x$ and $y$ are continuous functions of $t$ on an interval $I$, then the set of points $(x,y)$ obtained as $t$ varies over $I$ is called a parametric curve and is denoted by $C$.',
  'answer': 'True',
  'explanation': 'A parametric curve, also known as a plane curve, is defined by the set of points $(x,y)$ where $x = x(t)$ and $y = y(t)$ are continuous functions of $t$ over an interval $I$. As $t$ varies within this interval, the points $(x, y)$ trace out the curve $C$. For example, if $x(t) = \\cos(t)$ and $y(t) = \\sin(t)$ for $t \\in [0, 2\\pi]$, the parametric equations describe a circle.',
  'chapter': 'Parametric Equations and Polar Coordinates',
  'section': '1-1-parametric-equations',
  'number': '1'},
 {'question': "True or False? For the plane curve defined by the parametric equations $x = x(t)$ and $y = y(t)$, if both $x'(t)$ and $y'(t)$ exist and $x'(t) \\neq 0$, then the derivative $\\\\frac{dy}{dx}$ is given by $\\\\frac{dy}{dt} \\\\over \

In [9]:
all_questions =  dict()
for q in questions:
    chapter = q['chapter']
    section = q['section']
    quest = q['question'].replace("\\\\", "\\")
    explanation = q['explanation'].replace("\\\\", "\\")
    generated_question = f"""\\item {quest.replace("True or False:", "").strip()}
    
    \\ifnum \\Solutions=1 {{\\color{{EmphBlue}} Answer: {q['answer']} \\\\ Explanation: {explanation}}}
    \\fi"""

    if (chapter not in all_questions):
        all_questions[chapter] = dict()
    if (section not in all_questions[chapter]):
        all_questions[chapter][section] = []
    all_questions[chapter][section].append(generated_question)
        

In [23]:
location = "./generated_files/MultivariableCalculus/"
i = 0;
for chapter in all_questions:
    i+=1;
    # chapter_directory = location + f"Chapter {i}: " + chapter + "/"
    chapter_directory = location + f"Chapter{i}/"
    if (not os.path.exists(chapter_directory)):
        os.mkdir(chapter_directory)
    for section in all_questions[chapter]:
        path = chapter_directory + section + ".tex"
        section_title = ' '.join(section.split("-")[2:]).title()
        joined_questions = ('\n'.join(all_questions[chapter][section]))
        with open(path, "w") as f:
            latex = "\\section{" + section_title + "}" + f"""
                \\begin{{enumerate}}
                {unicode_to_latex(joined_questions, non_ascii_only=True)}
                \\end{{enumerate}}
                """
            f.write(latex)

In [21]:
i = 0
for a in all_questions:
    i+=1
    print("\\chapter{" + a + "}")
    for b in all_questions[a]:
        print("\\input{Chapter" + str(i) + f"/{b}" + "}")

\chapter{Parametric Equations and Polar Coordinates}
\input{Chapter1/1-1-parametric-equations}
\input{Chapter1/1-2-calculus-of-parametric-curves}
\input{Chapter1/1-3-polar-coordinates}
\input{Chapter1/1-4-area-and-arc-length-in-polar-coordinates}
\input{Chapter1/1-5-conic-sections}
\chapter{Vectors in Space}
\input{Chapter2/2-1-vectors-in-the-plane}
\input{Chapter2/2-2-vectors-in-three-dimensions}
\input{Chapter2/2-3-the-dot-product}
\input{Chapter2/2-4-the-cross-product}
\input{Chapter2/2-5-equations-of-lines-and-planes-in-space}
\input{Chapter2/2-6-quadric-surfaces}
\input{Chapter2/2-7-cylindrical-and-spherical-coordinates}
\chapter{Vector-Valued Functions}
\input{Chapter3/3-1-vector-valued-functions-and-space-curves}
\input{Chapter3/3-2-calculus-of-vector-valued-functions}
\input{Chapter3/3-3-arc-length-and-curvature}
\input{Chapter3/3-4-motion-in-space}
\chapter{Differentiation of Functions of Several Variables}
\input{Chapter4/4-1-functions-of-several-variables}
\input{Chapter4/4-

<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_2879630/1059457583.py:4: SyntaxWarning: invalid escape sequence '\c'
  print("\chapter{" + a + "}")
